In [1]:
from qiskit_ibm_runtime.fake_provider import FakeKyiv, FakeKyoto
from qiskit import QuantumCircuit
from qiskit.circuit.library import QFT, EfficientSU2
from qiskit.circuit import Instruction, Parameter

from colorama import Fore
import numpy as np


from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import AerSimulator
from qiskit.compiler import transpile
from qiskit.circuit.library import QFT
from qiskit_ibm_runtime.fake_provider import FakeMelbourne
from qiskit.quantum_info import hellinger_distance, hellinger_fidelity
from utils import connections_backend, connections_circuit



In [2]:
backend_kyiv = FakeKyiv()
backend_kyoto= FakeKyoto()

In [3]:
qc = QFT(4)
pm_kyiv = generate_preset_pass_manager(optimization_level=1, backend=backend_kyiv)
circuit_isa_kyiv = pm_kyiv.run(qc)

pm_kyoto = generate_preset_pass_manager(optimization_level=1, backend=backend_kyoto)
circuit_isa_kyoto = pm_kyoto.run(qc)



In [4]:
grader_operation_check_message : str = 'Basis gate check passed!'
def grader_operation_check(backend : any, circuit : QuantumCircuit):

    """
    Takes a backend and a circuit and see whether the gates in the circuit are supported by the backend

    Parameters : 
                backend : Target backend
                circuit : Quantum Circuit whose gates are to be checked with the gates in the target backend
    """

    op_names_backend : list[str] = backend.operation_names
    op_names_circuit_dict : dict[str,int]  = circuit.count_ops().items()
    #use set.
    for keys, val in op_names_circuit_dict:
        if keys in op_names_backend:
            None
        else : 
            print(Fore.RED + f'The backend does not support {keys} gate')

    return print(Fore.GREEN + grader_operation_check_message)

In [5]:
grader_operation_check(backend_kyiv, circuit_isa_kyiv)


Basis gate check passed!


In [6]:
grader_operation_check(backend_kyiv, circuit_isa_kyoto)


Basis gate check passed!


In [7]:
grader_connection_check_correct_message = 'The connections match with the backend'
grader_connection_check_incorrect_message = 'The connections does not match with the backend'

def grader_connection_check(backend: any, circuit : QuantumCircuit)-> None:

    """"
    Takes in a Quantum Circuit and a Backend and checks whether the conections in the circuit matches 
    with the topology of the backend. 

    Parameters : 
                backend : Target backend in which the circuit must fit.
                circuit : Circuit whose connections are to be matched with the backend

    Returns : Message : Whether the circuit fits in the topology of the backend. 
    """
    conn_backend = connections_backend(backend)
    conn_circ = connections_circuit(circuit)

    gate_conn_set_backend : dict[str,set] = {}
    for keys,items in conn_backend.items():
        gate_conn_set_backend[keys] = set(items)

    gate_conn_set_circuit : dict[str,set] = {}
    for keys,items in conn_circ.items():
        gate_conn_set_circuit[keys] = set(items)

    check_connections : list[int] = []
    for keys, items in gate_conn_set_circuit.items():
        check_connections.append(int(gate_conn_set_circuit[keys].issubset(gate_conn_set_backend[keys])))

    if 0 in check_connections:
        print(Fore.RED + grader_connection_check_incorrect_message)

    else : 
        print(Fore.GREEN +grader_connection_check_correct_message )

In [8]:
grader_connection_check(backend_kyiv, circuit_isa_kyiv)


The connections match with the backend


In [9]:
grader_connection_check(backend_kyiv, circuit_isa_kyoto)


The connections does not match with the backend


In [11]:
simulator = AerSimulator(method = "statevector")
backend_melbourne = FakeMelbourne()

In [12]:
def counts(circuit : QuantumCircuit, backend = simulator) -> dict[str, int]:
    """
    Function to add measurements to a provided circuit and run it on the Aer Simulator.

    Parameters 
        circuit : Quantum Circuit 
        backend : default = AerSimulator

    Returns  
        counts in a dict.

    """

    if 'measure' in circuit.count_ops():
        None
    else : 
        circuit.measure_active()
    result = simulator.run(circuit, backend = simulator).result()
    counts = result.get_counts()

    return counts

In [13]:
correct_fidelity_message = 'Congratualtions! Your circuit is within the given tolerance of the original circuit'
incorrect_fidelity_message = 'Oops! Your circuit is not within the given tolerance of the original circuit\n Try again!'


def grader_circuit_accuracy(circuit_target: QuantumCircuit, circuit_test : QuantumCircuit, 
                            backend : any = simulator, ε : float = 0.75):
    """
    Takes the test circuit and compares it with the target circuit.
    It computes the Hellinger distance and Hellinger Fidelity of the probability 
    distribution of the outputs of the to circuits and checks whether they are in
    the tolerance level.

    Parameters : 
        circuit_target :'QuantumCircuit' : Original Quantum Circuit
        circuit_test : 'QuantumCircuit': Transpiled Circuit.
        epsilon : Tolerance. default = 0.75
    Output :
        Prints out whether the transpiled circuit is within epsilon accuracy of the original circuit.
    """

    circuit_target = transpile(circuit_target, backend = simulator)
    circuit_test = transpile(circuit_test, backend = simulator)

    counts_target = counts(circuit_target)
    counts_test = counts(circuit_test)

    hell_dist : float = hellinger_distance(counts_target, counts_test)
    hell_fid : float = hellinger_fidelity(counts_target, counts_test)

    if hell_fid >= ε or hell_dist <= 1-ε:
        print(Fore.GREEN + correct_fidelity_message)

    else :
        print(Fore.RED + incorrect_fidelity_message)
    
    # return hell_dist, hell_fid


In [14]:
circuit_target = QFT(4)

circuit_target = transpile(circuit_target, backend = simulator)
circuit_test = transpile(circuit_target, backend = backend_melbourne)

circuit_esu2 = EfficientSU2(num_qubits=4)
np.random.seed(42)
params = np.random.random(circuit_esu2.num_parameters)
circuit_esu2 = circuit_esu2.assign_parameters(params)

In [15]:
grader_circuit_accuracy(circuit_target, circuit_test)

Congratualtions! Your circuit is within the given tolerance of the original circuit


In [16]:
grader_circuit_accuracy(circuit_target, circuit_esu2)

Oops! Your circuit is not within the given tolerance of the original circuit
 Try again!
